In [1]:
import os
import dill
import numpy as np
from fastai.tabular.all import *
from fastai.vision.all import *
from fastai.vision.utils import get_image_files
from Ambrosia import pre_process_image

from skimage import io

In [2]:
# this function only describes how much a singular value in al ist stands out.
# if all values in the lsit are high or low this is 1
# the smaller the proportiopn of number of disimilar vlaues are to other more similar values the lower this number
# the larger the gap between the dissimilar numbers and the simialr number the smaller this number
# only able to interpret probabilities or values between 0 and 1
# this function outputs an estimate an inverse of the classification confidence based on the probabilities of all the classes.
# the wedge threshold splits the data on a threshold with a magnitude of a positive int to force a ledge/peak in the data
def unkown_prob_calc(probs, wedge_threshold, wedge_magnitude=1, wedge=True):
    if wedge:
        probs = np.where(probs<=wedge_threshold , probs**(2*wedge_magnitude), probs)
        probs = np.where(probs>=wedge_threshold , probs**(1/(2*wedge_magnitude)), probs)
    diff_matrix = np.abs(probs[:, np.newaxis] - probs)
    diff_matrix_sum = np.sum(diff_matrix)
    probs_sum = np.sum(probs)
    class_val = (diff_matrix_sum/probs_sum)
    max_class_val = ((len(probs)-1)*2)
    kown_prob = class_val/max_class_val
    unknown_prob = 1-kown_prob
    return(unknown_prob)

In [3]:
wd = "/blue/hulcr/gmarais/Beetle_data/Beetle_classification_deep_data/Coccotypes_dactyliperda/Vial_16296/Subset_1"
image_name = "IMG_0052.JPG"
image_path = wd+"/"+image_name




# load compound image
image_arr = io.imread(image_path)
# Segment image into smaller images
pre_process = pre_process_image(manual_thresh_buffer=0.15, image = image_arr) # use image_dir if directory of image used
pre_process.segment(cluster_num=2, 
                    image_edge_buffer=50)
# load model
learn = load_learner(Path("/blue/hulcr/gmarais/Beetle_classifier/Models") / 'beetle_classifier.pkl', cpu=False, pickle_module=dill)
# get class names
class_arr = np.append(np.array(learn.dls.vocab), "Unknown")
# get predictions for all segments
compound_prob_arr = np.arry([])
for i in pre_process.col_image_lst:
    prob_ar = np.array(learn.predict(pre_process.col_image_lst[i])[2])
    unkown_prob = unkown_prob_calc(probs=pred, wedge_threshold=0.9, wedge_magnitude=1, wedge=True)
    prob_ar = np.append(prob_ar, unkown_prob)
    compound_prob_arr = np.append(compound_prob_arr, unkown_prob)

AttributeError: module 'numpy' has no attribute 'arry'

In [ ]:
compound_prob_arr